In [38]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [39]:
header_list = ['Date', 'User', 'Retry', 'Time', 'Returncode', 'Command']
data = pd.read_csv('../data/slurm_wrapper_ce5.log', names=header_list, sep=' - ', nrows=2000)
display(data['Returncode'].unique())
data

array(['returncode 0', 'returncode 1'], dtype=object)

,Date,User,Retry,Time,Returncode,Command
0,2020-10-16 08:15:39.278699,user 0,retry 0,time 0.07347559928894043,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
1,2020-10-16 08:18:08.313309,user 0,retry 0,time 0.18363237380981445,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
2,2020-10-16 08:22:48.128689,user 0,retry 0,time 0.07547116279602051,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
3,2020-10-16 08:25:13.257408,user 0,retry 0,time 0.09484362602233887,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
4,2020-10-16 08:31:01.460723,user 0,retry 0,time 0.07498788833618164,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
...,...,...,...,...,...,...
1995,2020-10-16 10:35:11.974730,user 9201,retry 0,time 0.40140223503112793,returncode 0,"command ['/usr/bin/scancel', '24996240']"
1996,2020-10-16 10:35:11.995461,user 9201,retry 0,time 0.40942907333374023,returncode 0,"command ['/usr/bin/scancel', '24996260']"
1997,2020-10-16 10:35:12.022998,user 9201,retry 0,time 0.33249592781066895,returncode 0,"command ['/usr/bin/scancel', '24996261']"
1998,2020-10-16 10:35:13.580876,user 9201,retry 0,time 0.13752985000610352,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."


In [40]:
data = data.drop(columns='Retry')
data = data[data['Returncode'] != 'returncode 0']
data

,Date,User,Time,Returncode,Command
357,2020-10-16 08:39:50.885933,user 9201,time 0.033915042877197266,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '..."
358,2020-10-16 08:39:50.906175,user 9201,time 0.01980280876159668,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '..."
359,2020-10-16 08:39:50.926236,user 9201,time 0.019769668579101562,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '..."
450,2020-10-16 08:50:40.257763,user 9201,time 0.02902054786682129,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '..."
451,2020-10-16 08:50:40.280953,user 9201,time 0.022747278213500977,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '..."
...,...,...,...,...,...
1765,2020-10-16 10:27:54.680816,user 9203,time 0.023977041244506836,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '..."
1766,2020-10-16 10:27:54.736298,user 9203,time 0.055191993713378906,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '..."
1946,2020-10-16 10:33:51.751891,user 9201,time 0.02063465118408203,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '..."
1947,2020-10-16 10:33:51.772029,user 9201,time 0.019786596298217773,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '..."


In [46]:
comm_regex = r"command ['/usr/bin/squeue\."
data = data[data['Command'] == comm_regex]
data

,Date,User,Time,Returncode,Command
